In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('./data_raw/base-des-lieux-et-des-equipements-culturels.csv', sep=';', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73234 entries, 0 to 73233
Data columns (total 47 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   source                                   73203 non-null  object 
 1   nom                                      73234 non-null  object 
 2   adresse                                  41249 non-null  object 
 3   complement_adresse                       335 non-null    object 
 4   code_postal                              72294 non-null  float64
 5   libelle_geographique                     73131 non-null  object 
 6   code_insee                               73233 non-null  object 
 7   code_insee_arrondt                       73068 non-null  object 
 8   identifiant_origine                      67285 non-null  object 
 9   type_equipement_ou_lieu                  73234 non-null  object 
 10  label_et_appellation                     53885

In [33]:
colonnes_a_garder = ['n_departement', 'type_equipement_ou_lieu','fonction_1']
df_selectionne = df[colonnes_a_garder]
df_selectionne

,n_departement,type_equipement_ou_lieu,fonction_1
0,69,Monument,Préservation
1,69,Monument,Préservation
2,69,Monument,Préservation
3,69,Monument,Préservation
4,69,Monument,Préservation
...,...,...,...
73229,44,Cinéma,Diffusion
73230,44,Cinéma,Diffusion
73231,44,Cinéma,Diffusion
73232,44,Cinéma,Diffusion


In [34]:
result = df.groupby(['n_departement', 'type_equipement_ou_lieu', 'fonction_1']).size().reset_index(name='nombre')
result

,n_departement,type_equipement_ou_lieu,fonction_1,nombre
0,01,Bibliothèque,Diffusion,240
1,01,Centre culturel,Préservation,1
2,01,Cinéma,Diffusion,21
3,01,Conservatoire,Education et formation,5
4,01,Espace protégé,Préservation,5
...,...,...,...,...
1666,99,Lieu de mémoire,Préservation,6
1667,99,Monument,Préservation,21
1668,99,Musée,Préservation,15
1669,99,Service d'archives,Préservation,7


In [35]:
result.drop(columns=['type_equipement_ou_lieu'], inplace=True)

In [36]:
# df_pivot = result.pivot(index=['n_departement'], columns='fonction_1', values='nombre').reset_index()
# df_pivot.columns.name = None
# result = df_pivot
# result

In [48]:


culture_df = result.groupby(['n_departement'])['nombre'].sum().reset_index()
culture_df = culture_df.rename(columns={'n_departement': 'code_departement'})

culture_df

,code_departement,nombre
0,01,708
1,02,828
2,03,836
3,04,363
4,05,336
...,...,...
99,972,189
100,973,123
101,974,330
102,976,21


In [44]:
population_df = pd.read_csv("./donnees/population_departement.csv")


In [49]:
result_merged = pd.merge(culture_df, population_df, on='code_departement', how='left')
result_merged

,code_departement,nombre,nom_departement,total_population
0,01,708,Ain,671937.0
1,02,828,Aisne,522791.0
2,03,836,Allier,332443.0
3,04,363,Alpes-de-Haute-Provence,166654.0
4,05,336,Hautes-Alpes,139942.0
...,...,...,...,...
99,972,189,NaN,NaN
100,973,123,NaN,NaN
101,974,330,NaN,NaN
102,976,21,NaN,NaN


In [55]:
result_merged['culture/1000POP'] = result_merged['nombre'] / result_merged['total_population'] * 1000


colonnes_a_garder = ['code_departement','culture/1000POP']
result_merged = result_merged[colonnes_a_garder]

,code_departement,culture/1000POP
0,01,1.053670
1,02,1.583807
2,03,2.514717
3,04,2.178166
4,05,2.400995
...,...,...
99,972,NaN
100,973,NaN
101,974,NaN
102,976,NaN


In [ ]:
sortie = './donnees/culture.csv'
result_merged.to_csv(sortie, index=False)